## Understanding Query Routing in RAG Applications
### What is Query Routing?
- Query routing (also called intelligent routing or query classification) is the process of dynamically directing queries to different retrieval systems, knowledge bases, or processing pipelines based on the query's characteristics, intent, or domain. Instead of using a one-size-fits-all approach, query routing ensures each query gets handled by the most appropriate specialized system.
### When is Query Routing Done?
Query routing occurs at the very beginning of the RAG pipeline:

- Query Analysis → Classify query type, intent, and domain
- 🔄 Query Routing → Direct to appropriate knowledge base/pipeline
- Specialized Retrieval → Use domain-specific retrievers
- Generation → LLM uses optimally retrieved context
- Optional Fallback → Route to general system if specialized fails

### Why Query Routing Matters
Different types of queries benefit from different retrieval strategies and knowledge sources:

- Technical queries → Code repositories, API docs
- Company policies → Internal knowledge bases
- General knowledge → Web search or general corpus
- Real-time data → Live APIs or databases
0 Personal queries → User-specific data stores

## Key Differences in Output Quality
### Without Query Routing:

- Generic Results: All queries use the same general knowledge base
- Lower Relevance: Documents lack domain-specific expertise
- Missed Opportunities: Specialized knowledge bases remain underutilized
- Inconsistent Quality: No optimization for different query types

### With Query Routing:

- Specialized Results: Each query gets directed to the most appropriate knowledge base
- Higher Precision: Domain-specific documents with expert-level content
- Optimal Resource Utilization: Right knowledge base for the right query
- Context-Aware Responses: Results tailored to query intent and user needs

## Technical Implementation Details

In [ ]:
# Approach 1 : LLM Based Classification
def classify_query_with_llm(query):
    prompt = f"""
    Classify this query into one of these categories:
    - technical_code: Programming, API, implementation questions
    - company_policy: Internal policies, procedures, HR matters
    - general_knowledge: General information, research, concepts
    - personal_data: User-specific information, personal projects
    - it_support: Technical support, troubleshooting
    
    Query: "{query}"
    
    Return: category, confidence (0-1), reasoning
    """
    response = llm.generate(prompt)
    return parse_classification(response)

In [ ]:
# Approach 2: Embedding based Routing 
def route_by_embedding_similarity(query_embedding, kb_embeddings):
    similarities = {}
    for kb_name, kb_embedding in kb_embeddings.items():
        similarity = cosine_similarity(query_embedding, kb_embedding)
        similarities[kb_name] = similarity
    
    # Route to KB with highest similarity above threshold
    best_kb = max(similarities, key=similarities.get)
    if similarities[best_kb] > 0.7:
        return best_kb
    else:
        return "general"  # Fallback

In [ ]:
#Approach 3 : Rule based Routing 

routing_rules = {
    "code_knowledge_base": [
        "implement", "code", "API", "function", "class", "python", "javascript"
    ],
    "hr_knowledge_base": [
        "policy", "company", "employee", "benefits", "remote work", "vacation"
    ],
    "it_knowledge_base": [
        "password", "reset", "login", "portal", "account", "troubleshoot"
    ],
    "user_data_store": [
        "my", "personal", "project", "deadline", "schedule", "task"
    ]
}

def route_by_keywords(query):
    query_lower = query.lower()
    scores = {}
    
    for kb, keywords in routing_rules.items():
        score = sum(1 for keyword in keywords if keyword in query_lower)
        scores[kb] = score / len(keywords)  # Normalize
    
    best_kb = max(scores, key=scores.get)
    return best_kb if scores[best_kb] > 0.2 else "general"

In [ ]:
# Approach 4: Hybrid Routing Strategies 

class HybridQueryRouter:
    def __init__(self):
        self.llm_classifier = LLMClassifier()
        self.embedding_router = EmbeddingRouter()
        self.keyword_router = KeywordRouter()
    
    def route_query(self, query):
        # Get predictions from all methods
        llm_result = self.llm_classifier.classify(query)
        embedding_result = self.embedding_router.route(query)
        keyword_result = self.keyword_router.route(query)
        
        # Combine with weights based on confidence
        if llm_result.confidence > 0.9:
            return llm_result.route
        elif embedding_result.similarity > 0.8:
            return embedding_result.route
        elif keyword_result.score > 0.5:
            return keyword_result.route
        else:
            return "general"  # Safe fallback

### Advanced Routing Strategies

In [ ]:
# 1. Multi-Route with Fusion
#Instead of routing to a single knowledge base, query multiple relevant ones and combine results:
def multi_route_query(query, classification_scores):
    # Route to top 2-3 knowledge bases
    top_kbs = sorted(classification_scores.items(), 
                    key=lambda x: x[1], reverse=True)[:3]
    
    all_results = []
    for kb_name, confidence in top_kbs:
        if confidence > 0.3:  # Threshold
            results = retrieve_from_kb(query, kb_name)
            # Weight results by routing confidence
            for result in results:
                result['routing_weight'] = confidence
            all_results.extend(results)
    
    # Combine and rerank
    return rerank_combined_results(all_results)

In [ ]:
#2. Hierarchical Routing
#Try specialized knowledge bases first, fall back to general:

def hierarchical_route(query):
    # Try specialized routing first
    specialized_route = classify_specialized(query)
    if specialized_route and specialized_route != "general":
        results = retrieve_from_kb(query, specialized_route)
        if len(results) > 0 and max(r['score'] for r in results) > 0.7:
            return results
    
    # Fallback to general knowledge base
    return retrieve_from_kb(query, "general")

In [ ]:
#3. User Context-Aware Routing
#Consider user profile and history:
def context_aware_route(query, user_context):

    base_classification = classify_query(query)
    
    # Adjust based on user context
    if user_context.role == "developer":
        if "code" in base_classification.top_categories:
            return "code_knowledge_base"
    elif user_context.role == "hr_manager":
        if "policy" in query.lower():
            return "hr_knowledge_base"
    
    # Consider user's recent query history
    recent_domains = get_user_recent_domains(user_context.user_id)
    if len(recent_domains) > 0:
        # Bias toward recently used domains
        adjusted_scores = adjust_scores_by_history(
            base_classification.scores, recent_domains
        )
        return max(adjusted_scores, key=adjusted_scores.get)
    
    return base_classification.best_route

### Performance Considerations
#### 1. Latency Optimization

- Parallel Classification: Run multiple routing methods simultaneously
- Caching: Cache routing decisions for similar queries
- Fast Fallback: Quick decision when confidence is low

#### 2. Quality Metrics

- Routing Accuracy: How often queries go to the optimal KB
- Result Quality: Relevance scores in routed vs non-routed scenarios
- Coverage: Ensure all query types have appropriate routes

In [ ]:
import React, { useState } from 'react';
import { Search, Route, Database, Code, Building, Globe, User, ArrowRight, CheckCircle, AlertTriangle } from 'lucide-react';

const QueryRoutingDemo = () => {
  const [activeTab, setActiveTab] = useState('without');
  const [selectedQuery, setSelectedQuery] = useState(0);
  
  // Demo queries with different types and optimal routes
  const testQueries = [
    {
      text: "How do I implement OAuth authentication in our Python API?",
      type: "Technical/Code",
      optimalRoute: "code_knowledge_base",
      confidence: 0.92,
      keywords: ["implement", "OAuth", "authentication", "Python", "API"],
      intent: "How-to programming question"
    },
    {
      text: "What is our company's remote work policy for new employees?",
      type: "Company Policy",
      optimalRoute: "hr_knowledge_base",
      confidence: 0.95,
      keywords: ["company", "remote work", "policy", "new employees"],
      intent: "Internal policy inquiry"
    },
    {
      text: "What are the latest trends in artificial intelligence research?",
      type: "General Knowledge",
      optimalRoute: "web_search",
      confidence: 0.88,
      keywords: ["latest", "trends", "artificial intelligence", "research"],
      intent: "Current information request"
    },
    {
      text: "Show me my project deadlines for this month",
      type: "Personal Data",
      optimalRoute: "user_data_store",
      confidence: 0.96,
      keywords: ["my", "project", "deadlines", "this month"],
      intent: "Personal information retrieval"
    },
    {
      text: "How do I reset my password for the company portal?",
      type: "IT Support",
      optimalRoute: "it_knowledge_base",
      confidence: 0.94,
      keywords: ["reset", "password", "company portal"],
      intent: "Technical support request"
    }
  ];

  // Knowledge bases with their specializations
  const knowledgeBases = {
    general: {
      name: "General Knowledge Base",
      icon: <Database className="w-5 h-5" />,
      color: "gray",
      description: "Mixed content from all domains",
      strengths: ["Broad coverage", "Fallback option"],
      weaknesses: ["Lower precision", "Generic responses"]
    },
    code_knowledge_base: {
      name: "Code & API Documentation",
      icon: <Code className="w-5 h-5" />,
      color: "blue",
      description: "Programming guides, API docs, code examples",
      strengths: ["Technical accuracy", "Code examples", "Best practices"],
      weaknesses: ["Limited to technical content"]
    },
    hr_knowledge_base: {
      name: "HR & Company Policies",
      icon: <Building className="w-5 h-5" />,
      color: "green",
      description: "Company policies, procedures, benefits",
      strengths: ["Official policies", "Up-to-date procedures"],
      weaknesses: ["Company-specific only"]
    },
    web_search: {
      name: "Web Search",
      icon: <Globe className="w-5 h-5" />,
      color: "purple",
      description: "Real-time web information",
      strengths: ["Current information", "Broad coverage"],
      weaknesses: ["Quality varies", "May be unreliable"]
    },
    user_data_store: {
      name: "Personal Data Store",
      icon: <User className="w-5 h-5" />,
      color: "orange",
      description: "User-specific data, projects, preferences",
      strengths: ["Personalized", "User context"],
      weaknesses: ["Limited scope"]
    },
    it_knowledge_base: {
      name: "IT Support Knowledge",
      icon: <Route className="w-5 h-5" />,
      color: "red",
      description: "Technical support, troubleshooting",
      strengths: ["Step-by-step guides", "Common issues"],
      weaknesses: ["Support-focused only"]
    }
  };

  // Demo documents in different knowledge bases
  const documents = {
    general: [
      {
        id: "g1",
        title: "General Authentication Overview",
        content: "Authentication is the process of verifying user identity. Various methods exist including passwords, tokens, and biometric verification.",
        relevance: [0.45, 0.25, 0.35, 0.15, 0.40],
        quality: "Generic"
      },
      {
        id: "g2",
        title: "Company Policies Summary",
        content: "Most companies have policies covering work arrangements, benefits, and procedures. Policies vary by organization size and industry.",
        relevance: [0.20, 0.50, 0.30, 0.10, 0.25],
        quality: "Generic"
      }
    ],
    code_knowledge_base: [
      {
        id: "c1",
        title: "OAuth 2.0 Implementation in Python Flask",
        content: "Complete guide to implementing OAuth 2.0 in Python Flask applications using Flask-OAuthlib. Includes code examples for authorization flows and token management.",
        relevance: [0.95, 0.15, 0.20, 0.10, 0.25],
        quality: "High Technical"
      },
      {
        id: "c2", 
        title: "Python API Authentication Best Practices",
        content: "Security best practices for Python APIs including JWT tokens, rate limiting, and secure session management with detailed implementation examples.",
        relevance: [0.88, 0.12, 0.18, 0.08, 0.30],
        quality: "High Technical"
      }
    ],
    hr_knowledge_base: [
      {
        id: "h1",
        title: "Remote Work Policy - New Employee Guidelines",
        content: "Our company supports flexible remote work arrangements. New employees are eligible for remote work after 90 days. Equipment and home office setup guidelines included.",
        relevance: [0.15, 0.92, 0.20, 0.25, 0.18],
        quality: "Official Policy"
      },
      {
        id: "h2",
        title: "Employee Benefits and Workplace Policies",
        content: "Comprehensive guide to employee benefits including health insurance, vacation time, professional development, and workplace conduct policies.",
        relevance: [0.12, 0.85, 0.18, 0.20, 0.22],
        quality: "Official Policy"
      }
    ],
    web_search: [
      {
        id: "w1",
        title: "AI Research Trends 2024: Latest Developments",
        content: "Recent breakthroughs in AI include large multimodal models, improved reasoning capabilities, and advances in robotics. Key trends shaping the future of AI research.",
        relevance: [0.25, 0.18, 0.95, 0.15, 0.20],
        quality: "Current/Timely"
      },
      {
        id: "w2",
        title: "Emerging AI Technologies and Research Directions",
        content: "Latest research in artificial intelligence focuses on interpretability, efficiency, and real-world applications. Current trends in neural architecture search and AutoML.",
        relevance: [0.22, 0.15, 0.90, 0.12, 0.18],
        quality: "Current/Timely"
      }
    ],
    user_data_store: [
      {
        id: "u1",
        title: "Your Project Timeline - Mobile App Development",
        content: "Project deadline: March 15th. Current tasks: UI design review (due March 5), backend API integration (due March 10), testing phase (March 12-14).",
        relevance: [0.10, 0.15, 0.12, 0.96, 0.18],
        quality: "Personal/Specific"
      },
      {
        id: "u2",
        title: "Q1 Deliverables and Milestones",
        content: "Your Q1 projects include client presentation (March 20), code review completion (March 8), and team retrospective (March 25). All deadlines confirmed.",
        relevance: [0.08, 0.12, 0.10, 0.94, 0.15],
        quality: "Personal/Specific"
      }
    ],
    it_knowledge_base: [
      {
        id: "i1",
        title: "Company Portal Password Reset Guide",
        content: "To reset your company portal password: 1) Go to login page, 2) Click 'Forgot Password', 3) Enter your employee email, 4) Check email for reset link, 5) Create new password following security requirements.",
        relevance: [0.20, 0.25, 0.15, 0.18, 0.95],
        quality: "Step-by-step"
      },
      {
        id: "i2",
        title: "Common IT Issues and Solutions",
        content: "Troubleshooting guide for common IT problems including password resets, VPN connections, software installations, and hardware issues. Contact IT for escalation.",
        relevance: [0.18, 0.22, 0.12, 0.15, 0.88],
        quality: "Step-by-step"
      }
    ]
  };

  const currentQuery = testQueries[selectedQuery];
  const queryIndex = selectedQuery;

  // Get results without routing (using general knowledge base only)
  const withoutRouting = documents.general.map(doc => ({
    ...doc,
    source: "general",
    score: doc.relevance[queryIndex]
  })).sort((a, b) => b.score - a.score);

  // Get results with routing (using optimal knowledge base)
  const withRouting = () => {
    const optimalKB = currentQuery.optimalRoute;
    const docs = documents[optimalKB] || documents.general;
    return docs.map(doc => ({
      ...doc,
      source: optimalKB,
      score: doc.relevance[queryIndex]
    })).sort((a, b) => b.score - a.score);
  };

  const routedResults = withRouting();

  const getKBInfo = (kbKey) => knowledgeBases[kbKey] || knowledgeBases.general;

  const ScoreBar = ({ score, color = "blue" }) => (
    <div className="w-full bg-gray-200 rounded-full h-2 mt-1">
      <div 
        className={`bg-${color}-500 h-2 rounded-full transition-all duration-300`} 
        style={{ width: `${score * 100}%` }}
      />
    </div>
  );

  const DocumentCard = ({ doc, rank, showSource = false }) => {
    const kbInfo = getKBInfo(doc.source);
    return (
      <div className="border rounded-lg p-4 mb-3 bg-white shadow-sm hover:shadow-md transition-shadow">
        <div className="flex justify-between items-start mb-2">
          <div className="flex items-center gap-2">
            <span className="text-lg font-bold text-gray-600">#{rank}</span>
            <div className="flex-1">
              <h3 className="font-semibold text-gray-800">{doc.title}</h3>
              <div className="flex gap-2 mt-1">
                <span className="text-xs bg-gray-100 px-2 py-1 rounded text-gray-600">
                  {doc.quality}
                </span>
                {showSource && (
                  <span className={`text-xs bg-${kbInfo.color}-100 text-${kbInfo.color}-700 px-2 py-1 rounded flex items-center gap-1`}>
                    {kbInfo.icon}
                    {kbInfo.name}
                  </span>
                )}
              </div>
            </div>
          </div>
          <div className="text-right w-32">
            <div className="text-sm font-medium">Relevance</div>
            <div className="text-lg font-bold">{doc.score.toFixed(3)}</div>
            <ScoreBar score={doc.score} color={kbInfo.color} />
          </div>
        </div>
        <p className="text-gray-600 text-sm leading-relaxed">{doc.content}</p>
      </div>
    );
  };

  const RoutingAnalysis = () => (
    <div className="bg-white rounded-lg shadow-sm p-6 mb-6">
      <div className="flex items-center gap-2 mb-4">
        <Route className="text-blue-500" />
        <h3 className="text-lg font-semibold">Query Routing Analysis</h3>
      </div>
      
      <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
        <div>
          <h4 className="font-medium mb-3">Query Classification:</h4>
          <div className="space-y-2">
            <div className="flex justify-between">
              <span className="text-sm text-gray-600">Query Type:</span>
              <span className="text-sm font-medium">{currentQuery.type}</span>
            </div>
            <div className="flex justify-between">
              <span className="text-sm text-gray-600">Intent:</span>
              <span className="text-sm font-medium">{currentQuery.intent}</span>
            </div>
            <div className="flex justify-between">
              <span className="text-sm text-gray-600">Confidence:</span>
              <span className="text-sm font-medium">{(currentQuery.confidence * 100).toFixed(0)}%</span>
            </div>
          </div>
        </div>
        
        <div>
          <h4 className="font-medium mb-3">Routing Decision:</h4>
          <div className="space-y-2">
            <div className="flex items-center gap-2">
              <span className="text-sm text-gray-600">Optimal Route:</span>
              <div className={`flex items-center gap-1 text-sm bg-${getKBInfo(currentQuery.optimalRoute).color}-100 text-${getKBInfo(currentQuery.optimalRoute).color}-700 px-2 py-1 rounded`}>
                {getKBInfo(currentQuery.optimalRoute).icon}
                {getKBInfo(currentQuery.optimalRoute).name}
              </div>
            </div>
            <div className="text-xs text-gray-500 mt-2">
              Key indicators: {currentQuery.keywords.slice(0, 3).join(", ")}
            </div>
          </div>
        </div>
      </div>
    </div>
  );

  return (
    <div className="max-w-7xl mx-auto p-6 bg-gray-50 min-h-screen">
      <h1 className="text-3xl font-bold text-center mb-2">RAG Query Routing Demo</h1>
      
      {/* Query Selection */}
      <div className="mb-6">
        <h2 className="text-lg font-semibold mb-3 text-center">Choose a Query to See Routing in Action:</h2>
        <div className="grid grid-cols-1 gap-3">
          {testQueries.map((query, idx) => (
            <button
              key={idx}
              onClick={() => setSelectedQuery(idx)}
              className={`p-4 rounded-lg border-2 text-left transition-all ${
                selectedQuery === idx
                  ? 'border-blue-500 bg-blue-50'
                  : 'border-gray-200 bg-white hover:border-gray-300'
              }`}
            >
              <div className="flex justify-between items-start">
                <div className="flex-1">
                  <div className="font-medium text-gray-800 mb-1">
                    "{query.text}"
                  </div>
                  <div className="flex gap-2 text-xs">
                    <span className="bg-gray-100 text-gray-700 px-2 py-1 rounded">
                      {query.type}
                    </span>
                    <span className={`bg-${getKBInfo(query.optimalRoute).color}-100 text-${getKBInfo(query.optimalRoute).color}-700 px-2 py-1 rounded flex items-center gap-1`}>
                      {getKBInfo(query.optimalRoute).icon}
                      {getKBInfo(query.optimalRoute).name}
                    </span>
                  </div>
                </div>
                <div className="text-right ml-4">
                  <div className="text-sm font-medium text-gray-600">
                    {(query.confidence * 100).toFixed(0)}% confidence
                  </div>
                </div>
              </div>
            </button>
          ))}
        </div>
      </div>

      {/* Current Query Display */}
      <div className="text-center mb-6">
        <div className="bg-blue-100 p-3 rounded-lg inline-block max-w-4xl">
          <Search className="inline mr-2" size={18} />
          <span className="font-medium">Active Query: "{currentQuery.text}"</span>
        </div>
      </div>

      {/* Routing Analysis */}
      <RoutingAnalysis />

      {/* Method Tabs */}
      <div className="flex justify-center mb-6">
        <div className="bg-white rounded-lg p-1 shadow-sm">
          <button
            onClick={() => setActiveTab('without')}
            className={`px-4 py-2 rounded-md mr-2 transition-colors flex items-center gap-2 ${
              activeTab === 'without' 
                ? 'bg-gray-500 text-white' 
                : 'text-gray-600 hover:bg-gray-100'
            }`}
          >
            <AlertTriangle size={16} />
            No Routing (General KB)
          </button>
          <button
            onClick={() => setActiveTab('with')}
            className={`px-4 py-2 rounded-md mr-2 transition-colors flex items-center gap-2 ${
              activeTab === 'with' 
                ? 'bg-green-500 text-white' 
                : 'text-gray-600 hover:bg-gray-100'
            }`}
          >
            <CheckCircle size={16} />
            With Smart Routing
          </button>
          <button
            onClick={() => setActiveTab('comparison')}
            className={`px-4 py-2 rounded-md mr-2 transition-colors flex items-center gap-2 ${
              activeTab === 'comparison' 
                ? 'bg-purple-500 text-white' 
                : 'text-gray-600 hover:bg-gray-100'
            }`}
          >
            <ArrowRight size={16} />
            Side-by-Side
          </button>
          <button
            onClick={() => setActiveTab('knowledge-bases')}
            className={`px-4 py-2 rounded-md transition-colors flex items-center gap-2 ${
              activeTab === 'knowledge-bases' 
                ? 'bg-blue-500 text-white' 
                : 'text-gray-600 hover:bg-gray-100'
            }`}
          >
            <Database size={16} />
            Knowledge Bases
          </button>
        </div>
      </div>

      {/* Results Display */}
      {activeTab === 'without' && (
        <div className="bg-white rounded-lg shadow-sm p-6">
          <div className="flex items-center gap-2 mb-4">
            <AlertTriangle className="text-gray-500" />
            <h2 className="text-xl font-semibold text-gray-600">
              Results: No Routing (General Knowledge Base Only)
            </h2>
          </div>
          <div className="bg-red-50 border border-red-200 rounded p-3 mb-4">
            <p className="text-red-800 text-sm">
              <strong>Limitation:</strong> Using only a general knowledge base provides generic, 
              lower-quality results that lack domain-specific expertise and up-to-date information.
            </p>
          </div>
          {withoutRouting.map((doc, index) => (
            <DocumentCard 
              key={doc.id} 
              doc={doc} 
              rank={index + 1} 
              showSource={true}
            />
          ))}
        </div>
      )}

      {activeTab === 'with' && (
        <div className="bg-white rounded-lg shadow-sm p-6">
          <div className="flex items-center gap-2 mb-4">
            <CheckCircle className="text-green-500" />
            <h2 className="text-xl font-semibold text-green-600">
              Results: Smart Query Routing
            </h2>
          </div>
          <div className="bg-green-50 border border-green-200 rounded p-3 mb-4">
            <p className="text-green-800 text-sm">
              <strong>Advantage:</strong> Query routed to {getKBInfo(currentQuery.optimalRoute).name} - 
              provides specialized, high-quality results with domain expertise and optimal relevance.
            </p>
          </div>
          {routedResults.map((doc, index) => (
            <DocumentCard 
              key={doc.id} 
              doc={doc} 
              rank={index + 1} 
              showSource={true}
            />
          ))}
        </div>
      )}

      {activeTab === 'comparison' && (
        <div className="grid grid-cols-1 lg:grid-cols-2 gap-6">
          <div className="bg-white rounded-lg shadow-sm p-6">
            <div className="flex items-center gap-2 mb-4">
              <AlertTriangle className="text-gray-500" />
              <h3 className="text-lg font-semibold text-gray-600">No Routing</h3>
            </div>
            <div className="mb-4 text-sm text-gray-600">
              Source: General Knowledge Base Only
            </div>
            {withoutRouting.map((doc, index) => (
              <div key={doc.id} className="border rounded p-3 mb-2">
                <div className="font-medium text-sm">#{index + 1} {doc.title}</div>
                <div className="text-gray-600 text-xs">Score: {doc.score.toFixed(3)}</div>
                <div className="text-xs text-gray-500">{doc.quality}</div>
              </div>
            ))}
            <div className="mt-3 p-2 bg-red-50 rounded text-xs text-red-700">
              ❌ Generic, low-relevance results
            </div>
          </div>
          
          <div className="bg-white rounded-lg shadow-sm p-6">
            <div className="flex items-center gap-2 mb-4">
              <CheckCircle className="text-green-500" />
              <h3 className="text-lg font-semibold text-green-600">Smart Routing</h3>
            </div>
            <div className="mb-4 text-sm text-gray-600">
              Source: {getKBInfo(currentQuery.optimalRoute).name}
            </div>
            {routedResults.map((doc, index) => (
              <div key={doc.id} className="border rounded p-3 mb-2">
                <div className="font-medium text-sm">#{index + 1} {doc.title}</div>
                <div className="text-green-600 text-xs">Score: {doc.score.toFixed(3)}</div>
                <div className="text-xs text-gray-500">{doc.quality}</div>
              </div>
            ))}
            <div className="mt-3 p-2 bg-green-50 rounded text-xs text-green-700">
              ✅ Specialized, high-relevance results
            </div>
          </div>
        </div>
      )}

      {activeTab === 'knowledge-bases' && (
        <div className="bg-white rounded-lg shadow-sm p-6">
          <h3 className="text-xl font-semibold mb-4">Available Knowledge Bases</h3>
          <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-4">
            {Object.entries(knowledgeBases).map(([key, kb]) => (
              <div key={key} className={`border-2 rounded-lg p-4 ${
                currentQuery.optimalRoute === key ? `border-${kb.color}-500 bg-${kb.color}-50` : 'border-gray-200'
              }`}>
                <div className="flex items-center gap-2 mb-2">
                  <div className={`text-${kb.color}-600`}>{kb.icon}</div>
                  <h4 className="font-semibold">{kb.name}</h4>
                  {currentQuery.optimalRoute === key && (
                    <span className="text-xs bg-green-100 text-green-700 px-2 py-1 rounded">OPTIMAL</span>
                  )}
                </div>
                <p className="text-sm text-gray-600 mb-3">{kb.description}</p>
                <div className="space-y-2">
                  <div>
                    <h5 className="text-xs font-medium text-green-700">Strengths:</h5>
                    <ul className="text-xs text-green-600">
                      {kb.strengths.map((strength, idx) => (
                        <li key={idx}>• {strength}</li>
                      ))}
                    </ul>
                  </div>
                  <div>
                    <h5 className="text-xs font-medium text-red-700">Limitations:</h5>
                    <ul className="text-xs text-red-600">
                      {kb.weaknesses.map((weakness, idx) => (
                        <li key={idx}>• {weakness}</li>
                      ))}
                    </ul>
                  </div>
                </div>
              </div>
            ))}
          </div>
        </div>
      )}

      {/* Performance Analysis */}
      <div className="mt-8 bg-yellow-50 border border-yellow-200 rounded-lg p-6">
        <h3 className="text-lg font-semibold text-yellow-800 mb-3">
          Impact Analysis for Query: "{currentQuery.text.substring(0, 60)}..."
        </h3>
        <div className="grid grid-cols-1 md:grid-cols-3 gap-6">
          <div>
            <h4 className="font-medium text-red-700 mb-2">Without Routing Issues:</h4>
            <ul className="space-y-1 text-sm text-red-600">
              <li>• Generic, non-specific responses</li>
              <li>• Lower relevance scores</li>
              <li>• Misses domain expertise</li>
              <li>• One-size-fits-all approach</li>
              <li>• May provide outdated information</li>
            </ul>
          </div>
          <div>
            <h4 className="font-medium text-green-700 mb-2">With Routing Benefits:</h4>
            <ul className="space-y-1 text-sm text-green-600">
              <li>• Specialized domain knowledge</li>
              <li>• Higher precision and accuracy</li>
              <li>• Context-appropriate responses</li>
              <li>• Up-to-date information</li>
              <li>• Optimized for query type</li>
            </ul>
          </div>
          <div>
            <h4 className="font-medium text-blue-700 mb-2">Routing Criteria:</h4>
            <ul className="space-y-1 text-sm text-blue-600">
              <li>• Query type classification</li>
              <li>• Domain keyword detection</li>
              <li>• Intent analysis</li>
              <li>• User context consideration</li>
              <li>• Confidence threshold validation</li>
            </ul>
          </div>
        </div>
      </div>

      {/* Implementation Details */}
      <div className="mt-6 bg-gray-100 rounded-lg p-6">
        <h3 className="text-lg font-semibold mb-3">Query Routing Implementation:</h3>
        <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
          <div>
            <h4 className="font-medium mb-2">Classification Methods:</h4>
            <ul className="space-y-2 text-sm">
              <li><strong>LLM-based:</strong> Use LLM to classify query intent and domain</li>
              <li><strong>Embedding similarity:</strong> Compare query embeddings to KB embeddings</li>
              <li><strong>Keyword matching:</strong> Rule-based routing using domain keywords</li>
              <li><strong>ML classification:</strong> Trained models for query categorization</li>
            </ul>
          </div>
          <div>
            <h4 className="font-medium mb-2">Routing Strategies:</h4>
            <ul className="space-y-2 text-sm">
              <li><strong>Single-route:</strong> Route to one optimal knowledge base</li>
              <li><strong>Multi-route:</strong> Query multiple KBs and combine results</li>
              <li><strong>Hierarchical:</strong> Try specialized first, fallback to general</li>
              <li><strong>Confidence-based:</strong> Route based on classification confidence</li>
            </ul>
          </div>
        </div>
      </div>
    </div>
  );
};

export default QueryRoutingDemo;